# Install and Import Qiskit
This section will help you install Qiskit (if needed) and import its main modules.

In [1]:
# If running in an environment where Qiskit is not installed, uncomment the next line:
!pip install qiskit[all,visualization] qiskit-ibm-runtime qiskit-aer tabulate matplotlib numpy

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import PassManager
import time
from tabulate import tabulate
from qiskit.circuit.random import random_circuit
from qiskit.transpiler.passes import (
    RemoveResetInZeroState,
    RemoveDiagonalGatesBeforeMeasure,
    Optimize1qGates,
    CommutationAnalysis,
    CommutativeCancellation,
    Collect2qBlocks,
    ConsolidateBlocks,
    UnitarySynthesis,
)

# Check Qiskit Version
Print the installed Qiskit version to confirm the environment setup.

In [2]:
import qiskit
print("Qiskit version:", qiskit.__version__)

Qiskit version: 2.1.0


# Create the service provider
Create a service provider instance to manage the quantum backend and other configurations.

In [3]:
# Initialize the Qiskit Runtime Service
service = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="ibm-q/open/main",  # ="<IBM Cloud CRN or instance name>", # Optionally specify the instance to use.
    token="ff85fe47849322f7afe26d0983205dfe03f5f14a2fd87d9a7d321fee1ca1b20f2442eec2d61cf900bace24e6c7484ae0e89ef2da6bf3893a80ac371310d9d5c6",
)
# List all available backends
backends = service.backends()
print("Available backends:")
for backend in backends:
    print(f"- {backend.name} ({backend.status})")

C:\Users\Nemanja\AppData\Local\Temp\ipykernel_16292\3052438554.py:2: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService(


Available backends:
- ibm_brisbane (<bound method IBMBackend.status of <IBMBackend('ibm_brisbane')>>)
- ibm_sherbrooke (<bound method IBMBackend.status of <IBMBackend('ibm_sherbrooke')>>)


# Preprocessing Quantum Circuits with Transpiler Passes

This cell defines a function `preprocess_circuit` that applies a sequence of Qiskit transpiler passes to optimize and simplify a given quantum circuit. The function uses passes such as removing resets in the zero state, eliminating diagonal gates before measurement, optimizing single-qubit gates, analyzing and canceling commutative gates, collecting two-qubit blocks, and consolidating blocks. Optionally, it can also synthesize unitary gates into a specified set of basis gates for further optimization.

In [4]:
def preprocess_circuit(qc: QuantumCircuit, basis_gates=None, basic_passes = True) -> QuantumCircuit:
    """
    This function serves as a preprocessing step for quantum circuits prior to transpilation, applying a series of transformation passes to optimize and simplify the circuit structure.

    :param qc: quantum  circuit
    :param basis_gates: basic gates
    :param basic_passes: basic passed
    :return:  quantum circuit
    """
    pm = PassManager()
    pm.append(RemoveResetInZeroState())
    pm.append(RemoveDiagonalGatesBeforeMeasure())
    pm.append(Optimize1qGates())
    pm.append(CommutationAnalysis())
    pm.append(CommutativeCancellation())
    pm.append(Collect2qBlocks())
    pm.append(ConsolidateBlocks())
    # The following passes are commented out as they may not be necessary for all circuits.
    # Uncomment them if you want to include them in your preprocessing pipeline.
    # RemoveBarriers removes barriers from the circuit, which can be useful for optimization.
    # CXCancellation removes controlled-X gates that are not needed, which can simplify the circuit.
    # RemoveUnusedRegisters removes any unused quantum or classical registers from the circuit.
    if not basic_passes:
        from qiskit.transpiler.passes import RemoveBarriers, Optimize1qGatesDecomposition, Collect1qRuns, CollectMultiQBlocks, CollectAndCollapse, CollectLinearFunctions, CollectCliffords, InverseCancellation, CommutativeInverseCancellation, Optimize1qGatesSimpleCommutation, RemoveFinalReset, HoareOptimizer, TemplateOptimization, ResetAfterMeasureSimplification, OptimizeCliffords, ElidePermutations, OptimizeAnnotated, Split2QUnitaries, RemoveIdentityEquivalent, ContractIdleWiresInControlFlow, OptimizeCliffordT
        pm.append(RemoveBarriers())
        pm.append(Optimize1qGatesDecomposition())
        pm.append(Collect1qRuns())
        pm.append(CollectMultiQBlocks())
        #pm.append(CollectAndCollapse())
        pm.append(CollectLinearFunctions())
        #pm.append(CollectCliffords())
        #pm.append(InverseCancellation())
        pm.append(CommutativeInverseCancellation())
        pm.append(Optimize1qGatesSimpleCommutation())
        pm.append(RemoveFinalReset())
        pm.append(HoareOptimizer())
        pm.append(TemplateOptimization())
        pm.append(ResetAfterMeasureSimplification())
        pm.append(OptimizeCliffords())
        pm.append(ElidePermutations())
        pm.append(OptimizeAnnotated())
        pm.append(Split2QUnitaries())
        pm.append(RemoveIdentityEquivalent())
        pm.append(ContractIdleWiresInControlFlow())
        pm.append(OptimizeCliffordT())

    # Optionally add UnitarySynthesis if basis_gates are provided
    # This pass synthesizes unitary gates into the specified basis gates.
    # It can be useful for optimizing the circuit further, especially if you have a specific gate
    # set you want to target for execution.
    # If you don't specify basis_gates, it will use the default basis gates of the backend.
    if basis_gates:
        pm.append(UnitarySynthesis(basis_gates=basis_gates))
    return pm.run(qc)

In [5]:
def test_ghz_circuit(n_qubits: int) -> QuantumCircuit:
    """Create large GHZ state with 20+ qubits"""
    qc = QuantumCircuit(n_qubits, name=f'Large_GHZ_{n_qubits}')

    # Create superposition on first qubit
    qc.h(0)

    # Entangle all qubits with the first one
    for i in range(1, n_qubits):
        qc.cx(0, i)

    print(f"✅ Created {n_qubits}-qubit large GHZ state")
    return qc

In [6]:
def test_create_deutsch_jozsa(n_qubits: int, oracle_type: str = 'constant') -> QuantumCircuit:
    """Create Deutsch-Jozsa algorithm circuit"""
    # n input qubits + 1 ancilla
    qc = QuantumCircuit(n_qubits + 1, n_qubits, name=f'Deutsch_Jozsa_{oracle_type}')

    # Initialize ancilla in |1⟩
    qc.x(n_qubits)

    # Apply Hadamard to all qubits
    for i in range(n_qubits + 1):
        qc.h(i)

    qc.barrier(label="Superposition")

    # Oracle implementation
    if oracle_type == 'constant_0':
        # Do nothing - f(x) = 0 for all x
        pass
    elif oracle_type == 'constant_1':
        # Flip ancilla - f(x) = 1 for all x
        qc.x(n_qubits)
    elif oracle_type == 'balanced':
        # Example balanced function: f(x) = x_0 ⊕ x_1 ⊕ ... (XOR of all inputs)
        for i in range(n_qubits):
            qc.cx(i, n_qubits)

    qc.barrier(label="Oracle")

    # Apply Hadamard to input qubits
    for i in range(n_qubits):
        qc.h(i)

    # Measure input qubits
    for i in range(n_qubits):
        qc.measure(i, i)

    print(f"✅ Created Deutsch-Jozsa algorithm ({oracle_type})")
    return qc

In [7]:
def test_create_qft(n_qubits: int = 3) -> QuantumCircuit:
    """Create Quantum Fourier Transform circuit"""
    qc = QuantumCircuit(n_qubits, name=f'QFT_{n_qubits}')

    def qft_rotations(circuit, n):
        """Apply the rotations for QFT"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi / 2 ** (n - qubit), qubit, n)
        qft_rotations(circuit, n)

    qft_rotations(qc, n_qubits)

    # Swap qubits to get correct order
    for qubit in range(n_qubits // 2):
        qc.swap(qubit, n_qubits - qubit - 1)

    print(f"✅ Created {n_qubits}-qubit QFT circuit")
    return qc

In [8]:
def test_build_random_test_circuit(n_qubits: int) -> QuantumCircuit:
    # depth=10 is arbitrary; adjust as needed
    qc = random_circuit(n_qubits, depth=10, max_operands=3, measure=True)#, seed=42)
    return qc

In [14]:
num_qubits = 50

backend = service.backend("ibm_brisbane")
seed_sim = 42
backend = AerSimulator.from_backend(backend, seed_simulator=seed_sim)
print(f"Backend: {backend.name}")
basis = backend._basis_gates()

# Create test circuits, please uncomment one that you want to test
# orig = test_ghz_circuit(num_qubits)
orig = test_build_random_test_circuit(num_qubits)
# orig = test_create_deutsch_jozsa(num_qubits)
# oring = test_create_qft(num_qubits)

# 1) Direct full transpile (optimization level 0)
start_time = time.time()
direct_0 = transpile(orig, backend, optimization_level=0)
print(f"Direct transpile opt level 0 took {time.time() - start_time:.3f} seconds")

# 2) Direct full transpile (optimization level 1)
start_time = time.time()
direct_1 = transpile(orig, backend, optimization_level=1)
print(f"Direct transpile opt level 1 took {time.time() - start_time:.3f} seconds")

# 3) Direct full transpile (optimization level 2)
start_time = time.time()
direct_2 = transpile(orig, backend, optimization_level=2)
print(f"Direct transpile opt level 2 took {time.time() - start_time:.3f} seconds")

# 4) Direct full transpile (optimization level 3)
start_time = time.time()
direct_3 = transpile(orig, backend, optimization_level=3)
print(f"Direct transpile opt level 3 took {time.time() - start_time:.3f} seconds")

# 5) Preprocess + transpile (optimization level 0)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_0 = transpile(pre, backend, optimization_level=0)
print(f"Preprocess + transpile 0 took {time.time() - start_time:.3f} seconds")

# 6) Preprocess + transpile (optimization level 1)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_1 = transpile(pre, backend, optimization_level=1)
print(f"Preprocess + transpile 1 took {time.time() - start_time:.3f} seconds")

# 7) Preprocess + transpile (optimization level 2)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_2 = transpile(pre, backend, optimization_level=2)
print(f"Preprocess + transpile 2 took {time.time() - start_time:.3f} seconds")

# 8) Preprocess + transpile (optimization level 3)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_3 = transpile(pre, backend, optimization_level=3)
print(f"Preprocess + transpile 3 took {time.time() - start_time:.3f} seconds")

# 4) Gather metrics
def metrics(circ):
    return {
        "depth": circ.depth(),
        "total_gates": len(circ.data),
        "total_ops": sum(circ.count_ops().values()),
        "cx_count": circ.count_ops().get("cx", 0),
        "h_count": circ.count_ops().get("h", 0),
        "x_count": circ.count_ops().get("x", 0),
        "measure_count": circ.count_ops().get("measure", 0),
        # "reset_count": circ.count_ops().get("reset", 0),
        # "barrier_count": circ.count_ops().get("barrier", 0),
        "swap_count": circ.count_ops().get("swap", 0),
        # "unitary_count": circ.count_ops().get("unitary", 0),
        # "reset_in_zero_count": circ.count_ops().get("reset_in_zero_state", 0),
        # "diagonal_gates_before_measure_count": circ.count_ops().get("diagonal_gates_before_measure", 0),
        # "optimize_1q_gates_count": circ.count_ops().get("optimize_1q_gates", 0),
        # "commutation_analysis_count": circ.count_ops().get("commutation_analysis", 0,
        # "commutative_cancellation_count": circ.count_ops().get("commutative_cancellation", 0),
        # "collect_2q_blocks_count": circ.count_ops().get("collect_2q_blocks", 0),
        # "consolidate_blocks_count": circ.count_ops().get("consolidate_blocks", 0),
        # "unitary_synthesis_count": circ.count_ops().get("unitary_synthesis", 0),
        "num_qubits": circ.num_qubits,
        "num_clbits": circ.num_clbits,
        "num_qubits_used": len(circ.qubits),
        "num_clbits_used": len(circ.clbits),
        "num_cregs_used": len(circ.cregs),
        "num_qregs_used": len(circ.qregs),
        # "num_parameters": len(circ.parameters),
        # "num_ancillas": len(circ.ancillas),
        # "num_ancillass": circ.num_ancillas,
    }

m_orig = metrics(orig)
m_direct_0 = metrics(direct_0)
m_direct_1 = metrics(direct_1)
m_direct_2 = metrics(direct_2)
m_direct_3 = metrics(direct_3)
m_after_pre_0 = metrics(after_pre_0)
m_after_pre_1 = metrics(after_pre_1)
m_after_pre_2 = metrics(after_pre_2)
m_after_pre_3 = metrics(after_pre_3)

# 5) Print comparison
print("\n--- Metrics Comparison ---")
headers = [
    "Metric",
    "Original",
    "Direct_0",
    "Direct_1",
    "Direct_2",
    "Direct_3",
    "Pre+0",
    "Pre+1",
    "Pre+2",
    "Pre+3"

]
rows = []
all_keys = m_orig.keys()
for key in all_keys:
    rows.append(
        [
            key,
            m_orig.get(key, ""),
            m_direct_0.get(key, ""),
            m_direct_1.get(key, ""),
            m_direct_2.get(key, ""),
            m_direct_3.get(key, ""),
            m_after_pre_0.get(key, ""),
            m_after_pre_1.get(key, ""),
            m_after_pre_2.get(key, ""),
            m_after_pre_3.get(key, "")
        ]
    )

Backend: aer_simulator_from(ibm_brisbane)
Direct transpile opt level 0 took 0.247 seconds
Direct transpile opt level 1 took 0.367 seconds
Direct transpile opt level 2 took 0.922 seconds
Direct transpile opt level 3 took 1.497 seconds
Preprocess + transpile 0 took 1.535 seconds
Preprocess + transpile 1 took 0.884 seconds
Preprocess + transpile 2 took 1.154 seconds
Preprocess + transpile 3 took 3.157 seconds

--- Metrics Comparison ---


In [15]:
print(tabulate(rows, headers=headers, tablefmt="grid"))

+-----------------+------------+------------+------------+------------+------------+---------+---------+---------+---------+
| Metric          |   Original |   Direct_0 |   Direct_1 |   Direct_2 |   Direct_3 |   Pre+0 |   Pre+1 |   Pre+2 |   Pre+3 |
+=================+============+============+============+============+============+=========+=========+=========+=========+
| depth           |         11 |       4772 |       2500 |       1886 |       2321 |    3566 |    1673 |    1624 |    1357 |
+-----------------+------------+------------+------------+------------+------------+---------+---------+---------+---------+
| total_gates     |        337 |      50371 |      17051 |      16075 |      16606 |   29809 |    9328 |    8760 |    8646 |
+-----------------+------------+------------+------------+------------+------------+---------+---------+---------+---------+
| total_ops       |        337 |      50371 |      17051 |      16075 |      16606 |   29809 |    9328 |    8760 |    8646 |


In [11]:
# print("\n-- Directly Transpiled Circuit --")
# direct_0.draw()
# direct_1.draw()
# direct_2.draw()
# print("\n-- Preprocessed + Transpiled Circuit --")
# after_pre_0.draw()
# after_pre_1.draw()
# after_pre_2.draw()